In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community

In [ ]:
!pip install unstructured

In [ ]:
!pip install pdfminer

In [ ]:
!pip install pi_heif

In [ ]:
!pip install unstructured_inference

In [6]:
! pip install pdf2image

In [ ]:
!pip install pdfminer.six==20201018
!pip install unstructured==0.6.7


In [3]:
# Install Tesseract engine
!apt install tesseract-ocr -y

# Install Python bindings
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [6]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [20]:
from unstructured.partition.image import partition_image
elements = partition_image(filename="/content/test_data1.png")
full_text = "\n".join([element.text for element in elements if element.text])
print(full_text)


Barcode Label
Guru
Aloo Bhujiya |
Net Weight : 250g
Ingredients : Besan,
Batch : 1000120230424 Edible Oil, Sugar, Salt, MFG Date : 24/04/2023 Masala, Termeric MIP? 120/- tnctusing at Taxes) ‘owder, Poteto
Powder, Di its, Best Before 90 Days from MFG Date wt ryfrui
NUTRITIONAL FACTS PER 100 GM Energy KCAL : 101 KCAL
Protein : 0.38g
Carbohydrate : 0.56
FAT: 12.4g
10001 Sugar: 24.4g
Address Line 1. MG Road,Address Line 2,Vadodara,Gujarat-390020.
Contact: 9727955514 softwareketan@gmail.com
FSSAI No. : 0123456790


In [ ]:
!pip install langchain_groq

In [25]:
GROQ_API_KEY="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"

In [27]:
from langchain_groq import ChatGroq
model=ChatGroq(model_name="llama-3.1-8b-instant", api_key=GROQ_API_KEY)

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = """
You are an intelligent food label parser.

Below is raw text extracted from a packaged food product's label:

--- BEGIN TEXT ---
{label_text}
--- END TEXT ---

Extract and return the information in a clean JSON format with the following fields:
- brand
- product_name
- net_weight
- ingredients (as a list)
- manufacturing_date
- best_before
- mrp (if available)
- nutrition_per_100g: {{ energy_kcal, protein_g, carbohydrate_g, fat_g, sugar_g }}
- contact
- email
- fssai_no
- address

Respond with only valid JSON.
"""


prompt = PromptTemplate(
    input_variables=["label_text"],
    template=template
)

In [36]:
chain = LLMChain(llm=model, prompt=prompt)
response = chain.run(label_text=full_text)
print(response)

```json
{
  "brand": "Guru",
  "product_name": "Aloo Bhujiya",
  "net_weight": 250,
  "ingredients": [
    "Besan",
    "Edible Oil",
    "Sugar",
    "Salt",
    "Masala",
    "Termeric Powder",
    "Potato Powder",
    "Diet Soda"
  ],
  "manufacturing_date": "24/04/2023",
  "best_before": "90 Days from MFG Date",
  "mrp": null,
  "nutrition_per_100g": {
    "energy_kcal": 101,
    "protein_g": 0.38,
    "carbohydrate_g": 0.56,
    "fat_g": 12.4,
    "sugar_g": 24.4
  },
  "contact": "9727955514",
  "email": "softwareketan@gmail.com",
  "fssai_no": "0123456790",
  "address": "MG Road, Vadodara, Gujarat-390020"
}
```
Note: The contact number and email address are not typically included in a food label, but I have included them in the response as per your request. Also, I have corrected "Di its" to "Diet Soda" as it seems to be a typo in the provided text.


In [43]:
import json
import re

cleaned_response = re.sub(r"^```json|```$", "", response.strip(), flags=re.MULTILINE).strip()

json_end = cleaned_response.rfind("}")
if json_end != -1:
    cleaned_response = cleaned_response[:json_end+1]

try:
    product_info = json.loads(cleaned_response)
except json.JSONDecodeError as e:
    print(cleaned_response)
    raise e


In [49]:
from langchain.prompts import PromptTemplate

prompt2 = PromptTemplate(
    input_variables=["product_json"],
    template="""
You are a health analysis expert. Based on the product details in this JSON:

{product_json}

Perform the following:
1. Classify product type (e.g., snack, beverage, etc.).
2. Evaluate nutrition levels and whether they are high/low (fat, sugar, etc.).
3. Is this product healthy for general users?
4. Would you recommend this to diabetic or heart patients?
5. Are any claims misleading?

Respond in a structured JSON format.
""")

chain2 = LLMChain(llm=model, prompt=prompt2)


In [50]:
response2 = chain2.run(product_json=cleaned_response)

In [59]:

chain2 = LLMChain(llm=model, prompt=prompt2)

product_json_str = json.dumps(cleaned_response, indent=2)

response2 = chain2.run(product_json=product_json_str)

In [64]:
import json

try:
    response_data = json.loads(response2)
    print(json.dumps(response_data, indent=4, ensure_ascii=False))
except json.JSONDecodeError:
    print("Response is not valid JSON:")
    print(response2)


Response is not valid JSON:
**Health Analysis Report**

```json
{
  "product_name": "Aloo Bhujiya",
  "product_type": "snack",
  "nutrition_level": {
    "energy_kcal": "high",
    "protein_g": "low",
    "carbohydrate_g": "low",
    "fat_g": "high",
    "sugar_g": "very high"
  },
  "healthiness": "unhealthy",
  "recommended_for": {
    "diabetic_patients": "no",
    "heart_patients": "no"
  },
  "claims_misleading": "yes"
}
```

**Explanation**

1. **Product Type**: Based on the ingredients and product name, I classify the product as a snack.
2. **Nutrition Levels**: 
   - **Energy Kcal**: High (101 kcal per 100g)
   - **Protein G**: Low (0.38g per 100g)
   - **Carbohydrate G**: Low (0.56g per 100g)
   - **Fat G**: High (12.4g per 100g)
   - **Sugar G**: Very High (24.4g per 100g)
3. **Healthiness**: The high levels of sugar and fat in the product make it unhealthy for general users.
4. **Recommended for**: 
   - **Diabetic Patients**: No, due to the high sugar content.
   - **Heart 